# Cleaning the NOW corpus
This notebook shows the cleaning process that will be used for the ADA project. Here, only a sample of the data is used (from [here](https://www.corpusdata.org/now_corpus.asp)), but the methods should be the same once scaled to the full database.

The NOW database is composed of billions of words from online newspapers and magazines from 20 different countries. The data we downloaded comes in different files which can be used together or independently. These files are:

1. **now-samples-lexicon.txt**: this is the full dictionnary of the english language, a lexicon. It contains four clolumns, `wID` which is the word id, `word` the actual word, `lemma` which is family of the word (ie: if word is walked, lemma is walk) and `PoS` which is the part of speech.
2. **now-samples_sources.txt**: this is the source of every text, in order it contains the text id, the number of words, the date, the country, the website, the url and title of the article.
3. **text.txt**: this file has the complete texts of the articles, the first column is the `textID` in the format @@textID, the second column is the full text, complete with html paragraphs and headers. It is important to note that to prevent plagiarism, every 200 words, 10 words are replaced by the string "@ @ @ @ @ @ @ @ @ @". Combined words are also split, example "can't" is written as "ca n't" and punctuation is surrounded by spaces.
4. **wordLemPoS.txt**: finally, this file contains the `word`, `lemma` and `PoS` for each word in the texts, one by one, so one could read the texts by reading doewn the columns. Along with that is the `textID` from where the word is and an `ID (seq)` which we don't know what it does.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql import *
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

## text.txt

In [2]:
text_rdd = sc.textFile('sample_data/text.txt') \
            .filter(lambda r: len(r)>20) \

In [3]:
text_rows = text_rdd.map(lambda r: Row(text=r)) 
text = spark.createDataFrame(text_rows)

In [4]:
text = text.withColumn('textID', regexp_extract('text','(\\d+)',1))
text.show()

+--------------------+------+
|                text|textID|
+--------------------+------+
|@@11241 <p> Sol Y...| 11241|
|@@11242 <h> That ...| 11242|
|@@11243 <h> A sub...| 11243|
|@@11244 <h> Refle...| 11244|
|@@21242 <h> Ask A...| 21242|
|@@21243 <p> NEW Y...| 21243|
|@@31240 <p> IRELA...| 31240|
|@@31241 <h> Shaki...| 31241|
|@@31242 <p> ENTRE...| 31242|
|@@41240 <p> Syria...| 41240|
|@@41241 <h> Publi...| 41241|
|@@41244 <h> The B...| 41244|
|@@51243 <h> MPAA ...| 51243|
|@@61240 <h> Mum '...| 61240|
|@@61242 <h> IPPC ...| 61242|
|@@71240 <p> North...| 71240|
|@@71241 <h> James...| 71241|
|@@71242 <h> From ...| 71242|
|@@71243 <h> ' Inc...| 71243|
|@@71244 <h> Mary ...| 71244|
+--------------------+------+
only showing top 20 rows

